# <font color = "darkblue">Ecuaciones Diferenciales Parciales de Evolución </font>

# <font color = "darkblue"> Ecuación de Calor</font>

Las ecuaciones diferenciales parciales (EDPs) constituyen una área de suma importancia en la física, ya que modelan sistemas que varían con respecto a más de una variable independiente, por ejemplo, tanto el tiempo como el espacio. 


Del punto de vista numérico, el tipo de EDPs que es (conceptualmente) más sencillo son las llamadas **parabólicas**, es decir, **ecuaciones de evolución**, de las cuales la más conocida es la **ecuación de calor** o **ecuación de difusión.**

# Ecuación de Calor 

La ecuación de calor modela el esparcimiento en el tiempo y en el espacio de un "paquete" de calor (perturbación local de temperatura en una región) o un "paquete" de concentración de una sustancia física o química. 

Sea $u(t, \mathbf{x})$ la temperatura o la concentración de la sustancia en la posición **x** al tiempo t. Recordando que la ecuación de calor es 

$$\frac{\partial u(t, \mathbf{x})}{\partial t} = D \, \nabla^2 u(t, \mathbf{x}),$$


con  $\nabla^2 := \frac{\partial^2}{\partial x^2} + \frac{\partial^2}{\partial y^2} + \frac{\partial^2}{\partial z^2}$ en tres dimensiones.

Esta ecuación nos dice cómo varía la concentración en el tiempo, dadas las condiciones locales en el espacio. Se deriva en términos de una ley de conservación: 

$$\frac{\partial u}{\partial t} + \nabla \cdot \mathbf{J} = 0,$$

donde el flujo de calor o de concentración $\mathbf{J}$ es proporcional al gradiente local:

$$\mathbf{J} = -D \, \nabla u.$$


La ecuación de calor es una **ecuación de evolución** que describe cómo evoluciona el sistema en el tiempo. Por lo tanto, su tratamiento se sigue de forma directa de lo que sabemos para EDOs.

# Una dimensión 

El caso más sencillo, con sólo una dimensión espacial. En este caso, la ecuación de calor se reduce a 

$$\frac{\partial u(t, x)}{\partial t} = D \frac{\partial^2 u(t, x)}{\partial x^2}.$$

Para resolverla, necesitaremos además:
- una condición inicial $u(t=0, x) = f(x)$ (una función del espacio)
- condiciones en la frontera, $u(t, x)$ para todo $x$ en la frontera del dominio espacial, y para todo $t$.

Dado que, como siempre, no podemos resolver problemas de naturaleza continua en la computadora, debemos *aproximar* la solución $u(t, x)$ de alguna forma. La manera más sencilla es, de nuevo, utilizar una **discretización**.

¿Cómo se puede discretizar $u(t,x)$ utilizando un tamaño de paso h en el tiempo y k en el espacio? Pensando que los valores posibles de $x$ son en el intervalo $[0, L]$.

Denotando con $t_n$ el tiempo al paso número $n$, y con $u^n_i$ la aproximación de la solución en el nodo número $i$ en el espacio al tiempo $n$.



Primero hay que aproximar la derivada parcial temporal usando un paso de Euler, esto es
$$
\dfrac{\partial u(t,x)}{\partial t} \approx \dfrac{u(t+h,x) - u(t,x)}{h}
$$

Por otro lado la segunda parcial con respecto a x usando diferencias centradas es

$$
\dfrac{\partial^2 u(t,x)}{\partial x^2} \approx \dfrac{u(t,x+k)-2u(t,x)+u(t,x-k)}{k^2}
$$

Por lo que, si se sustituye en la expresión se tiene la aproximación para u
$$
u(t,x) \approx \dfrac{k^2 u(t+h,x) - hu(t,x+2k)+2hu(t,x+k)}{h+k^2}
$$

Por lo que sustituyendo en la ecuación de calor
$$
\dfrac{u(t+h,x) - u(t,x)}{h} \approx D\dfrac{u(t,x+k)-2u(t,x)+u(t,x-k)}{k^2} $$
$$\Rightarrow [u(t+h,x) - u(t,x)] \approx \dfrac{Dh}{k^2}[u(t,x+k)-2u(t,x)+u(t,x-k)]$$

$$\Rightarrow u(t+h,x) \approx \dfrac{Dh}{k^2}[u(t,x+k)-2u(t,x)+u(t,x-k)] +  u(t,x)\\
$$

**Ejercicio1** 

Considera la ecuación de calor en una dimensión sobre el intervalo de $x=-L$ a $x=L$, con condición inicial $u(t=0, x) = \delta(x)$, donde $\delta$ es la delta de Dirac, y condiciones de frontera absorbentes (de Dirichlet), es decir, $u(t, -L) = u(t, +L) = 0$ para todo $t > 0$.

(i) ¿Qué esperas intuitivamente que pase durante la evolución? ¿Qué ocurrirá para tiempos largos?

(ii) Implementa el sistema, tomando cuidado en lo que ocurre en las fronteras. Para hacerlo, utiliza un vector para representar el estado actual del sistema, y otro vector para el estado al tiempo siguiente.

(iii) Dibuja la evolución en el tiempo (es decir, dibuja $u(x,t)$ para varios valores de $t$ en una sola gráfica, y con `Interact`). ¿Ocurre lo que esperabas?

(iv) ¿Qué ocurre si tomas otra condición inicial, por ejemplo una suma de dos deltas?

**Punto Extra**

(v) ¿Qué pasa con la energía del sistema? 

In [ ]:
using Plots  
using Interact
include("herramientas.jl")
gr()

In [ ]:
t1, x1, u1 = heat_eq(10, 80, 0.01, 1., 1)


In [ ]:
@manipulate for t in 1:length(t1)
    plot(x1, u1[t], ylims=(0,1))    
end

In [ ]:
anim = @animate for t in 1:10:length(u1)
    plot(x1, u1[t], ylims=(0,1))
    end every 1

gif(anim, "Dirichlet.gif", fps=20)